In [1]:
# Cell 1: Import necessary libraries
import os
import sys
import torch
from torch.utils.data import DataLoader
import time
import random
import numpy as np
import yaml
from box import Box

# Add the parent directory to the sys.path
ProjectPath = os.path.dirname(os.path.dirname(os.path.abspath(os.getcwd())))
sys.path.append(ProjectPath)

# Add the directory containing the exp module to the sys.path
exp_module_path = os.path.join(ProjectPath, 'mrDiff')
sys.path.append(exp_module_path)

from mrDiff.exp.exp_main import Exp_Main


In [2]:
class Args:
    def __init__(self, config_filename, cmd_args=None):
        assert config_filename.endswith('.yml')
        self.config_filename = config_filename
        self.config = self.read_config()
        self.override_with_cmd_args(cmd_args)
        
    def read_config(self):
        with open(self.config_filename, 'r') as file:
            config = yaml.safe_load(file)
        return Box(config)
    
    def override_with_cmd_args(self, cmd_args):
        if cmd_args is not None:
            for key, value in vars(cmd_args).items():
                if value is not None:
                    self.config[key] = value

    def __getattr__(self, name):
        return getattr(self.config, name)

config_filename = '/home/liranc6/ecg_forecasting/mrDiff/configs/config.yml'
args = Args(config_filename)

print('Args in experiment:')
print(vars(args))

# Accessing nested attribute
try:
    print(args.hardware)
    print(args.hardware.gpu)
except AttributeError as e:
    print(e)

Args in experiment:
{'config_filename': '/home/liranc6/ecg_forecasting/mrDiff/configs/config.yml', 'config': Box({'general': {'random_seed': 2023, 'evaluate': False, 'tag': None, 'dataset': 'ETTm1', 'features': 'S', 'training_mode': 'ONE', 'interval': 1000}, 'optimization': {'learning_rate': 0.001, 'batch_size': 2048, 'test_batch_size': 32, 'patience': 10, 'weight_decay': 1e-05, 'lradj': '3', 'pct_start': 0.3}, 'hardware': {'num_workers': 0, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0'}, 'paths': {'checkpoints': './checkpoints/', 'train_data': '/home/liranc6/ecg_forecasting/mrDiff/datasets/SCINet_timeseries/ETT-data/ETT'}, 'data': {'freq': 'h', 'embed': 'timeF', 'cols': [], 'target': -1, 'inverse': False, 'individual': True, 'use_ar_init': False, 'use_residual': True, 'uncertainty': False, 'norm_method': 'z_score', 'normtype': 0}, 'training': {'iterations': {'itr': 1, 'pretrain_epochs': 2, 'train_epochs': 2, 'sample_times': 1}, 'identifiers': {'id_worst': -1, 'focu

In [3]:
print('Args in experiment:')
print(vars(args))

Args in experiment:
{'config_filename': '/home/liranc6/ecg_forecasting/mrDiff/configs/config.yml', 'config': Box({'general': {'random_seed': 2023, 'evaluate': False, 'tag': None, 'dataset': 'ETTm1', 'features': 'S', 'training_mode': 'ONE', 'interval': 1000}, 'optimization': {'learning_rate': 0.001, 'batch_size': 2048, 'test_batch_size': 32, 'patience': 10, 'weight_decay': 1e-05, 'lradj': '3', 'pct_start': 0.3}, 'hardware': {'num_workers': 0, 'use_gpu': True, 'gpu': 0, 'use_multi_gpu': False, 'devices': '0'}, 'paths': {'checkpoints': './checkpoints/', 'train_data': '/home/liranc6/ecg_forecasting/mrDiff/datasets/SCINet_timeseries/ETT-data/ETT'}, 'data': {'freq': 'h', 'embed': 'timeF', 'cols': [], 'target': -1, 'inverse': False, 'individual': True, 'use_ar_init': False, 'use_residual': True, 'uncertainty': False, 'norm_method': 'z_score', 'normtype': 0}, 'training': {'iterations': {'itr': 1, 'pretrain_epochs': 2, 'train_epochs': 2, 'sample_times': 1}, 'identifiers': {'id_worst': -1, 'focu

In [4]:
# Set use_gpu based on availability
args.config['use_gpu'] = torch.cuda.is_available()

# Handle multi-GPU settings
if args.config['use_gpu'] and args.config['hardware']['use_multi_gpu']:
    args.config['hardware']['devices'] = args.config['hardware']['devices'].replace(' ', '')
    device_ids = args.config['hardware']['devices'].split(',')
    args.config['hardware']['device_ids'] = [int(id_) for id_ in device_ids]
    args.config['hardware']['gpu'] = args.config['hardware']['device_ids'][0]
    args.config['optimization']['patience'] = 30
else:
    args.config['hardware']['device_ids'] = [0]  # Default to single GPU or CPU

# Random seed initialization
if 'random_seed' in args.config['general']:
    fix_seed = args.config['general']['random_seed']
    random.seed(fix_seed)
    torch.manual_seed(fix_seed)
    np.random.seed(fix_seed)

# Load the dataset
if args.config['general']['dataset'] == 'ETTm1':
    args.config['paths']['datasets_dir'] = '/home/liranc6/ecg_forecasting/mrDiff/datasets/SCINet_timeseries/ETT-data/ETT'
    args.config['data']['data'] = 'ETTm1'
    args.config['paths']['root_path'] = args.config['paths']['datasets_dir']
    args.config['paths']['data_path'] = 'ETTm1.csv'
    args.config['data']['num_vars'] = 7

if args.config['general']['features'] == "S":
    args.config['data']['num_vars'] = 1

In [5]:
import torch
import random
import numpy as np

# Initialize metrics lists
mae_, mse_, rmse_, mape_, mspe_, rse_, corr_, nrmse_ = [], [], [], [], [], [], [], []

# Ensure the necessary configurations are available
iterations = args.config.get('training', {}).get('iterations', {}).get('itr', 1)
random_seed = args.config.get('general', {}).get('random_seed', 2023)
dataset = args.config.get('general', {}).get('dataset', 'default_dataset')
model = args.config.get('training', {}).get('model', {}).get('model', 'default_model')
features = args.config.get('general', {}).get('features', 'S')
seq_len = args.config.get('training', {}).get('sequence', {}).get('seq_len', 336)
label_len = args.config.get('training', {}).get('sequence', {}).get('label_len', 336)
pred_len = args.config.get('training', {}).get('sequence', {}).get('pred_len', 96)
learning_rate = args.config.get('optimization', {}).get('learning_rate', 0.001)
batch_size = args.config.get('optimization', {}).get('batch_size', 64)
inverse = args.config.get('data', {}).get('inverse', False)
tag = args.config.get('general', {}).get('tag', None)

# sourcery skip: assign-if-exp
for ii in range(iterations):
    # setting record of experiments

    # random seed
    fix_seed = ii if iterations > 1 else random_seed

    random.seed(fix_seed)
    torch.manual_seed(fix_seed)
    np.random.seed(fix_seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True  # Can change it to False --> default: False
    torch.backends.cudnn.enabled = True

    setting = f"{model}_{dataset}_ft{features}_sl{seq_len}_ll{label_len}_pl{pred_len}_lr{learning_rate}_bs{batch_size}_inv{inverse}_itr{ii}"
    if tag is not None:
        setting += f"_{tag}"

    exp = Exp_Main(args)

    print(f'>>>>>>>start training : {setting}>>>>>>>>>>>>>>>>>>>>>>>>>>')
    exp.train(setting)

    print(f'>>>>>>>testing : {setting}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<')
    mae, mse, rmse, mape, mspe, rse, corr, nrmse = exp.test(setting, test=1)

    mae_.append(mae)
    mse_.append(mse)
    rmse_.append(rmse)
    mape_.append(mape)
    mspe_.append(mspe)
    rse_.append(rse)
    corr_.append(corr)
    nrmse_.append(nrmse)

    torch.cuda.empty_cache()

print('Final mean normed: ')
print('> mae:{:.4f}, std:{:.4f}'.format(np.mean(mae_), np.std(mae_)))
print('> mse:{:.4f}, std:{:.4f}'.format(np.mean(mse_), np.std(mse_)))
print('> rmse:{:.4f}, std:{:.4f}'.format(np.mean(rmse_), np.std(rmse_)))
print('> mape:{:.4f}, std:{:.4f}'.format(np.mean(mape_), np.std(mape_)))
print('> rse:{:.4f}, std:{:.4f}'.format(np.mean(rse_), np.std(rse_)))
print('> corr:{:.4f}, std:{:.4f}'.format(np.mean(corr_), np.std(corr_)))
print('> nrmse:{:.4f}, std:{:.4f}'.format(np.mean(nrmse_), np.std(nrmse_)))

Use GPU: cuda:0


>>>>>>>start training : DDPM_ETTm1_ftS_sl336_ll336_pl96_lr0.001_bs2048_invFalse_itr0>>>>>>>>>>>>>>>>>>>>>>>>>>
train 34129
val 11425
test 11425
Epoch: 1 cost time: 8.26150631904602
Epoch: 1, Steps: 16 | Train Loss: 0.9056404 Vali Loss: 0.0762225
Validation loss decreased (inf --> 0.076222).  Saving model ...
Updating learning rate to 0.001
Epoch: 2 cost time: 7.661164045333862
Epoch: 2, Steps: 16 | Train Loss: 0.6420200 Vali Loss: nan
Validation loss decreased (0.076222 --> nan).  Saving model ...
Updating learning rate to 0.001
>>>>>>>testing : DDPM_ETTm1_ftS_sl336_ll336_pl96_lr0.001_bs2048_invFalse_itr0<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
test 11425
loading model
Elapsed time: 30.75 ms
Elapsed time: 29.46 ms
Elapsed time: 31.14 ms
Elapsed time: 31.63 ms
Elapsed time: 31.44 ms


KeyboardInterrupt: 